In [1]:
from keras import models, layers
from keras.datasets import mnist
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from keras.optimizers import SGD

In [64]:
# load data
(train_X, train_y), (test_X, test_y) = mnist.load_data()
train_X.shape, test_X.shape

((60000, 28, 28), (10000, 28, 28))

In [38]:
model = models.Sequential()
# Set input shape in the first layer
# First convolutional layer with filter size (3,3) and 32 filters followed by a max pooling layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))

# Second convolutional layer with filter size (3,3) and 64 filters followed by a max pooling layer
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

#  Flatten to provide features to the classifier
model.add(layers.Flatten())

# Dense layer to interpret the features with 64 nodes
model.add(layers.Dense(64, activation='relu'))

# To handle overfitting
model.add(layers.Dropout(0.2))

# Output layer. Has 10 nodes for 0-9 outputs. Use a softmax activation to find probabilities and assign classes
model.add(layers.Dense(10, activation='softmax'))

In [39]:
# opt = SGD(lr=0.01, momentum=0.9) # Stochastic gradient descent optimizer with a learning rate of 0.01 and a momentum of 0.9
# opt = 'adam'
opt = 'rmsprop'
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [65]:
# Preprocessing of the images
# Reshape dataset to have 4D vector with the number of channels of the input image as 1 (since image is grayscale)
train_X = train_X.reshape((60000, 28, 28, 1))
test_X = test_X.reshape((10000, 28, 28, 1))

# Normalize by rescaling pixel values from range [0, 255] to [0, 1] by 1st converting the value to float and then by dividing from max value
train_X= train_X.astype('float32') / 255
test_X= test_X.astype('float32') / 255

# Convert to one-hot encoding
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

In [66]:
# Add noise to the dataset
noise_factor = 0.25
train_X_noisy = train_X + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=train_X.shape)
test_X_noisy = test_X + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=test_X.shape)

train_X_noisy = np.clip(train_X_noisy, 0., 1.)
test_X_noisy = np.clip(test_X_noisy, 0., 1.)

In [67]:
history = model.fit(train_X_noisy, train_y, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
750/750 [==============================] - 37s 49ms/step - loss: 0.1103 - accuracy: 0.9668 - val_loss: 0.0917 - val_accuracy: 0.9727
Epoch 2/5
750/750 [==============================] - 37s 49ms/step - loss: 0.0813 - accuracy: 0.9754 - val_loss: 0.0674 - val_accuracy: 0.9802
Epoch 3/5
750/750 [==============================] - 37s 49ms/step - loss: 0.0685 - accuracy: 0.9789 - val_loss: 0.0593 - val_accuracy: 0.9834
Epoch 4/5
750/750 [==============================] - 37s 49ms/step - loss: 0.0589 - accuracy: 0.9820 - val_loss: 0.0650 - val_accuracy: 0.9833
Epoch 5/5
750/750 [==============================] - 37s 49ms/step - loss: 0.0536 - accuracy: 0.9837 - val_loss: 0.0563 - val_accuracy: 0.9850


In [68]:
test_loss, test_acc = model.evaluate(test_X_noisy, test_y)
print('Accuracy:', test_acc)
print('Loss: ', test_loss)

313/313 [==============================] - 3s 9ms/step - loss: 0.0500 - accuracy: 0.9842
Accuracy: 0.9842000007629395
Loss:  0.05002663657069206
